In [43]:
import os
import random
import tensorflow as tf 
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from data_preprocess import *
from image_preprocess import *
from bcnn_model import *

In [44]:
class vgg_bilinear_model:
    def __init__(self, imgs, keep_prob, weights_file = None, weights_file_last = None, sess = None, finetune = False):
        self.finetune = finetune                     # train all layers or only last layer
        self.imgs = imgs                             # input img data
        self.keep_prob = keep_prob                   # fc dropout ratio
        self.last_layer_parameters = []              # param only last fc layer, normal train will use
        self.parameters = []                         # param whole net, fine tune will use
        self.weights_file = weights_file             # pretrained weights file path
        self.weights_file_last = weights_file_last   # last fc layer weights file path
        self.conv_layers()                           # model's conv part, here use vgg
        self.bilinear_layers()                       # model's bilinear part, feature fusion
        self.fc_layers()                             # model's classify part, full connect
    
    def conv_layers(self):
        # preprovess, why???
        #with tf.name_scope('preprocess') as scope:
        #    mean = tf.constant([123.68, 116.779, 103.939], dtype=tf.float32, shape=[1, 1, 1, 3], name='img_mean')
        #    images = self.imgs - mean
        #    print('Adding Data Augmentation')
            
        # -------------------------------------------------------------------------------------------------------- #
        
        # conv1_1
        with tf.variable_scope('conv1_1'):
            weights = tf.get_variable('W', [3, 3, 3, 64], initializer=tf.contrib.layers.xavier_initializer(), 
                                      trainable = self.finetune)
            biases = tf.get_variable('b', [64], initializer=tf.constant_initializer(0.1), trainable = self.finetune)
            conv = tf.nn.conv2d(self.imgs, weights, strides = [1, 1, 1, 1], padding = 'SAME')
            self.conv1_1 = tf.nn.relu(conv + biases)
            self.parameters += [weights, biases]
        
        # conv1_2
        with tf.variable_scope('conv1_2'):
            weights = tf.get_variable('W', [3, 3, 64, 64], initializer=tf.contrib.layers.xavier_initializer(), 
                                      trainable = self.finetune)
            biases = tf.get_variable('b', [64], initializer=tf.constant_initializer(0.1), trainable = self.finetune)
            conv = tf.nn.conv2d(self.conv1_1, weights, strides = [1, 1, 1, 1], padding = 'SAME')
            self.conv1_2 = tf.nn.relu(conv + biases)
            self.parameters += [weights, biases]
        
        # pool1
        self.pool1 = tf.nn.max_pool(self.conv1_2, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME', name = 'pool1')

        # -------------------------------------------------------------------------------------------------------- #

        # conv2_1
        with tf.variable_scope('conv2_1'):
            weights = tf.get_variable('W', [3, 3, 64, 128], initializer=tf.contrib.layers.xavier_initializer(), 
                                      trainable = self.finetune)
            biases = tf.get_variable('b', [128], initializer=tf.constant_initializer(0.1), trainable = self.finetune)
            conv = tf.nn.conv2d(self.pool1, weights, strides = [1, 1, 1, 1], padding = 'SAME')
            self.conv2_1 = tf.nn.relu(conv + biases)
            self.parameters += [weights, biases]
        
        # conv2_2
        with tf.variable_scope('conv2_2'):
            weights = tf.get_variable('W', [3, 3, 128, 128], initializer=tf.contrib.layers.xavier_initializer(), 
                                      trainable = self.finetune)
            biases = tf.get_variable('b', [128], initializer=tf.constant_initializer(0.1), trainable = self.finetune)
            conv = tf.nn.conv2d(self.conv2_1, weights, strides = [1, 1, 1, 1], padding = 'SAME')
            self.conv2_2 = tf.nn.relu(conv + biases)
            self.parameters += [weights, biases]
        
        # pool2
        self.pool2 = tf.nn.max_pool(self.conv2_2, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME', name = 'pool2')
        
        # -------------------------------------------------------------------------------------------------------- #

        # conv3_1
        with tf.variable_scope('conv3_1'):
            weights = tf.get_variable('W', [3, 3, 128, 256], initializer=tf.contrib.layers.xavier_initializer(), 
                                      trainable = self.finetune)
            biases = tf.get_variable('b', [256], initializer=tf.constant_initializer(0.1), trainable = self.finetune)
            conv = tf.nn.conv2d(self.pool2, weights, strides = [1, 1, 1, 1], padding = 'SAME')
            self.conv3_1 = tf.nn.relu(conv + biases)
            self.parameters += [weights, biases]
        
        # conv3_2
        with tf.variable_scope('conv3_2'):
            weights = tf.get_variable('W', [3, 3, 256, 256], initializer=tf.contrib.layers.xavier_initializer(), 
                                      trainable = self.finetune)
            biases = tf.get_variable('b', [256], initializer=tf.constant_initializer(0.1), trainable = self.finetune)
            conv = tf.nn.conv2d(self.conv3_1, weights, strides = [1, 1, 1, 1], padding = 'SAME')
            self.conv3_2 = tf.nn.relu(conv + biases)
            self.parameters += [weights, biases]
        
        # conv3_3
        with tf.variable_scope('conv3_3'):
            weights = tf.get_variable('W', [3, 3, 256, 256], initializer=tf.contrib.layers.xavier_initializer(), 
                                      trainable = self.finetune)
            biases = tf.get_variable('b', [256], initializer=tf.constant_initializer(0.1), trainable = self.finetune)
            conv = tf.nn.conv2d(self.conv3_2, weights, strides = [1, 1, 1, 1], padding = 'SAME')
            self.conv3_3 = tf.nn.relu(conv + biases)
            self.parameters += [weights, biases]
        
        # pool3
        self.pool3 = tf.nn.max_pool(self.conv3_3, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME', name = 'pool3')

        # -------------------------------------------------------------------------------------------------------- #

        # conv4_1
        with tf.variable_scope('conv4_1'):
            weights = tf.get_variable('W', [3, 3, 256, 512], initializer=tf.contrib.layers.xavier_initializer(), 
                                      trainable = self.finetune)
            biases = tf.get_variable('b', [512], initializer=tf.constant_initializer(0.1), trainable = self.finetune)
            conv = tf.nn.conv2d(self.pool3, weights, strides = [1, 1, 1, 1], padding = 'SAME')
            self.conv4_1 = tf.nn.relu(conv + biases)
            self.parameters += [weights, biases]
        
        # conv4_2
        with tf.variable_scope('conv4_2'):
            weights = tf.get_variable('W', [3, 3, 512, 512], initializer=tf.contrib.layers.xavier_initializer(), 
                                      trainable = self.finetune)
            biases = tf.get_variable('b', [512], initializer=tf.constant_initializer(0.1), trainable = self.finetune)
            conv = tf.nn.conv2d(self.conv4_1, weights, strides = [1, 1, 1, 1], padding = 'SAME')
            self.conv4_2 = tf.nn.relu(conv + biases)
            self.parameters += [weights, biases]
        
        # conv4_3
        with tf.variable_scope('conv4_3'):
            weights = tf.get_variable('W', [3, 3, 512, 512], initializer=tf.contrib.layers.xavier_initializer(), 
                                      trainable = self.finetune)
            biases = tf.get_variable('b', [512], initializer=tf.constant_initializer(0.1), trainable = self.finetune)
            conv = tf.nn.conv2d(self.conv4_2, weights, strides = [1, 1, 1, 1], padding = 'SAME')
            self.conv4_3 = tf.nn.relu(conv + biases)
            self.parameters += [weights, biases]
        
        # pool4
        self.pool4 = tf.nn.max_pool(self.conv4_3, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME', name = 'pool4')

        # -------------------------------------------------------------------------------------------------------- #

        # conv5_1
        with tf.variable_scope('conv5_1'):
            weights = tf.get_variable('W', [3, 3, 512, 512], initializer=tf.contrib.layers.xavier_initializer(), 
                                      trainable = self.finetune)
            biases = tf.get_variable('b', [512], initializer=tf.constant_initializer(0.1), trainable = self.finetune)
            conv = tf.nn.conv2d(self.pool4, weights, strides = [1, 1, 1, 1], padding = 'SAME')
            self.conv5_1 = tf.nn.relu(conv + biases)
            self.parameters += [weights, biases]
        
        # conv5_2
        with tf.variable_scope('conv5_2'):
            weights = tf.get_variable('W', [3, 3, 512, 512], initializer=tf.contrib.layers.xavier_initializer(), 
                                      trainable = self.finetune)
            biases = tf.get_variable('b', [512], initializer=tf.constant_initializer(0.1), trainable = self.finetune)
            conv = tf.nn.conv2d(self.conv5_1, weights, strides = [1, 1, 1, 1], padding = 'SAME')
            self.conv5_2 = tf.nn.relu(conv + biases)
            self.parameters += [weights, biases]
        
        # conv5_3
        with tf.variable_scope('conv5_3'):
            weights = tf.get_variable('W', [3, 3, 512, 512], initializer=tf.contrib.layers.xavier_initializer(), 
                                      trainable = self.finetune)
            biases = tf.get_variable('b', [512], initializer=tf.constant_initializer(0.1), trainable = self.finetune)
            conv = tf.nn.conv2d(self.conv5_2, weights, strides = [1, 1, 1, 1], padding = 'SAME')
            self.conv5_3 = tf.nn.relu(conv + biases)
            self.parameters += [weights, biases]
    
    def bilinear_layers(self):
        conv1 = tf.transpose(self.conv5_3, perm = [0, 3, 1, 2])
        conv1 = tf.reshape(conv1, [-1, 512, 196])
        
        conv2 = tf.transpose(self.conv5_3, perm = [0, 3, 1, 2])
        conv2 = tf.reshape(conv1, [-1, 512, 196])
        conv2 = tf.transpose(conv2, perm = [0, 2, 1])
        
        phi_I = tf.matmul(conv1, conv2)
        phi_I = tf.reshape(phi_I, [-1, 512 * 512])
        phi_I = tf.divide(phi_I, 196.0)
        
        y_ssqrt = tf.multiply(tf.sign(phi_I), tf.sqrt(tf.abs(phi_I) + 1e-12))
        
        z = tf.nn.l2_normalize(y_ssqrt, dim = 1)
        
        print('Shape of z', z.get_shape())
        
        self.bilinear_feature = z
    
    def fc_layers(self):
        with tf.variable_scope('fc-new') as scope:
            fc3w = tf.get_variable('W', [512 * 512, 12], initializer = tf.contrib.layers.xavier_initializer(), trainable = True)
            fc3b = tf.get_variable('b', [12], initializer=tf.constant_initializer(0.1), trainable = True)
            fc3l = tf.nn.bias_add(tf.matmul(self.bilinear_feature, fc3w), fc3b)
            self.fc3l = tf.nn.dropout(fc3l, self.keep_prob)
            self.last_layer_parameters += [fc3w, fc3b]
    
    # for first train use, load pretrained vgg weights 
    def load_vgg_weights(self, session):
        weights_dict = np.load(self.weights_file, encoding = 'bytes')
        vgg_layers = ['conv1_1',
                      'conv1_2',
                      'conv2_1',
                      'conv2_2',
                      'conv3_1',
                      'conv3_2',
                      'conv3_3',
                      'conv4_1',
                      'conv4_2',
                      'conv4_3',
                      'conv5_1',
                      'conv5_2',
                      'conv5_3']

        for op_name in vgg_layers:
            with tf.variable_scope(op_name, reuse = True):
                # biases
                var = tf.get_variable('b')
                print('Adding weights to',var.name)
                session.run(var.assign(weights_dict[op_name + '_b']))
                
                # weights
                var = tf.get_variable('W')
                print('Adding weights to',var.name)
                session.run(var.assign(weights_dict[op_name + '_W']))
    
    # for finetune train use, load last fc layer weights 
    def load_last_layer_weights(self, session):
        weights_dict = np.load(self.weights_file_last, encoding = 'bytes')
        
        for i, var in enumerate(self.last_layer_parameters):
            session.run(var.assign(weights_dict['arr_0'][i]))
            print('Adding weights to',var.name)
    
    # for predict use, load all trained weights 
    def load_all_own_weights(self, session):
        weights_dict_conv = np.load(self.weights_file, encoding = 'bytes')
        weights_dict_fc = np.load(self.weights_file_last, encoding = 'bytes')
        
        for i, var in enumerate(self.parameters):
            session.run(var.assign(weights_dict_conv['arr_0'][i]))
            print('Adding weights to',var.name)
            
        for i, var in enumerate(self.last_layer_parameters):
            session.run(var.assign(weights_dict_fc['arr_0'][i]))
            print('Adding weights to',var.name)

In [14]:
weights_file = "vgg16_weights.npz"
weights_file_last_layer = "bcnn_last_weights.npz"
weights_file_conv = "bcnn_conv_weights.npz"

batch_size = 32
lr = 0.001
finetune_step = -1

image_size = 224
n_classes = 12

epoch = 100

In [15]:
tf.reset_default_graph()
sess = tf.Session()
# build input
imgs = tf.placeholder(tf.float32, shape = [None, image_size, image_size, 3], name = "input_x")
targets = tf.placeholder(tf.int32, shape = [None, n_classes], name = "input_y")
keep_prob = tf.placeholder(tf.float32, name = "keep_prob")

# build cal graph
bilinear_cnn = vgg_bilinear_model(imgs, keep_prob, pretrained_weights = weights_file, sess = sess, finetune = False)

# build cal node parameter
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels = targets, logits = bilinear_cnn.fc3l)
cost = tf.reduce_mean(cross_entropy, name = "cost")
optimizer = tf.train.MomentumOptimizer(learning_rate=0.9, momentum=0.9).minimize(cost)

predicted = tf.nn.softmax(bilinear_cnn.fc3l, name = "predicted")
correct_pred = tf.equal(tf.argmax(predicted, 1), tf.argmax(targets, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name = "accuracy")

# start train
sess.run(tf.global_variables_initializer())
bilinear_cnn.load_vgg_weights(sess)

for v in tf.trainable_variables():
    print("Trainable variables", v)
print('Starting training')


###################################
train_img_batch, train_label_batch = get_batchs(tfrecord_filename = "train.tfrecords", 
                                                    image_size = image_size, classes = n_classes,
                                                    batch_size = batch_size, min_after_dequeue = 500)
test_img_batch, test_label_batch = get_batchs(tfrecord_filename = "test.tfrecords", 
                                                  image_size = image_size, classes = n_classes,
                                                  batch_size = batch_size, min_after_dequeue = 500)

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

for i in range(int((4512 / batch_size) * epoch)):
    # get data
    train_imgs, train_labels= sess.run([train_img_batch, train_label_batch])
    test_imgs, test_labels= sess.run([test_img_batch, test_label_batch])

    # train single loop
    sess.run(optimizer, feed_dict={imgs: train_imgs, targets: train_labels, keep_prob: 0.5})
    train_loss = sess.run(cost, feed_dict={imgs: train_imgs, targets: train_labels, keep_prob: 1.})
    print("train loss: %10f"%(train_loss))
            
    # print test stat
    if (i % 50 == 0):
        print("########### epoch %d ###########"%((i / (4512 / batch_size)) + 1))
        print("########### loop %d ###########"%(i))
        test_loss = sess.run(cost, feed_dict={imgs: test_imgs, targets: test_labels, keep_prob: 1.})
        test_acc = sess.run(accuracy, feed_dict={imgs: test_imgs, targets: test_labels, keep_prob: 1.})
        print("test loss: %10f   test accuracy: %10f"%(test_loss, test_acc))
        print("########### loop %d ###########"%(i))

# close data queue
coord.request_stop()
coord.join(threads)





sess.close()

Shape of z (?, 262144)
Adding weights to conv1_1/b:0
Adding weights to conv1_1/W:0
Adding weights to conv1_2/b:0
Adding weights to conv1_2/W:0
Adding weights to conv2_1/b:0
Adding weights to conv2_1/W:0
Adding weights to conv2_2/b:0
Adding weights to conv2_2/W:0
Adding weights to conv3_1/b:0
Adding weights to conv3_1/W:0
Adding weights to conv3_2/b:0
Adding weights to conv3_2/W:0
Adding weights to conv3_3/b:0
Adding weights to conv3_3/W:0
Adding weights to conv4_1/b:0
Adding weights to conv4_1/W:0
Adding weights to conv4_2/b:0
Adding weights to conv4_2/W:0
Adding weights to conv4_3/b:0
Adding weights to conv4_3/W:0
Adding weights to conv5_1/b:0
Adding weights to conv5_1/W:0
Adding weights to conv5_2/b:0
Adding weights to conv5_2/W:0
Adding weights to conv5_3/b:0
Adding weights to conv5_3/W:0
Trainable variables <tf.Variable 'fc-new/W:0' shape=(262144, 12) dtype=float32_ref>
Trainable variables <tf.Variable 'fc-new/b:0' shape=(12,) dtype=float32_ref>
Starting training
train loss:   2.38

train loss:   2.059854
train loss:   2.166144
train loss:   2.145414
train loss:   2.029237
train loss:   1.859474
train loss:   2.216050
train loss:   1.743692
train loss:   1.630572
train loss:   2.050435
train loss:   1.835695
train loss:   1.831742
train loss:   2.020522
train loss:   1.979112
train loss:   1.856297
train loss:   1.962246
train loss:   1.786938
train loss:   2.096676
train loss:   1.962884
train loss:   1.897461
train loss:   1.781772
train loss:   1.810292
train loss:   1.771865
train loss:   1.657670
train loss:   1.678895
train loss:   1.990570
########### epoch 3 ###########
########### loop 300 ###########
test loss:   1.808197   test accuracy:   0.437500
########### loop 300 ###########
train loss:   1.979552
train loss:   1.878313
train loss:   1.723810
train loss:   1.975907
train loss:   1.726971
train loss:   1.779817
train loss:   1.673483
train loss:   1.811248
train loss:   1.732351
train loss:   1.987812
train loss:   1.989836
train loss:   1.793826
t

train loss:   1.475223
train loss:   1.748424
train loss:   1.854350
train loss:   1.728695
train loss:   1.653807
train loss:   1.647550
train loss:   1.696574
########### epoch 5 ###########
########### loop 600 ###########
test loss:   1.924795   test accuracy:   0.343750
########### loop 600 ###########
train loss:   1.610030
train loss:   1.776258
train loss:   1.868029
train loss:   1.784459
train loss:   2.009964
train loss:   1.673656
train loss:   1.796841
train loss:   1.832717
train loss:   1.693540
train loss:   1.665277
train loss:   1.543631
train loss:   1.784191
train loss:   1.883344
train loss:   1.546617
train loss:   1.709311
train loss:   1.466413
train loss:   1.641239
train loss:   1.488118
train loss:   1.652622
train loss:   1.510592
train loss:   1.627752
train loss:   1.711871
train loss:   1.742021
train loss:   1.623832
train loss:   1.770029
train loss:   1.724753
train loss:   1.599714
train loss:   1.599489
train loss:   1.384137
train loss:   1.879089
t

train loss:   1.707645
train loss:   1.981596
train loss:   1.615044
train loss:   1.703029
train loss:   1.579448
train loss:   1.260110
train loss:   1.984511
train loss:   1.680921
train loss:   1.478911
train loss:   1.595568
train loss:   1.688447
train loss:   1.508180
train loss:   1.432497
train loss:   1.499534
train loss:   1.691497
train loss:   1.436104
train loss:   1.664922
train loss:   1.696616
train loss:   1.599530
train loss:   1.788674
train loss:   1.581938
train loss:   1.331281
train loss:   1.532670
train loss:   1.725927
train loss:   1.450769
train loss:   1.775030
train loss:   1.796053
train loss:   1.272418
train loss:   1.662580
train loss:   1.601976
train loss:   1.420797
train loss:   1.404541
train loss:   1.692815
train loss:   1.685374
train loss:   1.640227
train loss:   1.761330
train loss:   1.594606
train loss:   1.586344
train loss:   1.438642
train loss:   1.530664
train loss:   1.484700
train loss:   1.824511
train loss:   1.245412
train loss:

train loss:   1.489740
train loss:   1.219646
train loss:   1.320525
train loss:   1.492426
train loss:   1.459769
train loss:   1.719492
train loss:   1.234712
train loss:   1.399006
train loss:   1.261828
train loss:   1.343151
train loss:   1.267620
train loss:   1.369664
train loss:   1.492986
train loss:   1.422379
train loss:   1.501049
train loss:   1.570751
train loss:   1.195089
train loss:   1.519068
train loss:   1.617379
train loss:   1.529949
train loss:   1.777970
train loss:   1.523162
train loss:   1.379591
train loss:   1.498520
train loss:   1.374900
train loss:   1.281172
train loss:   1.627275
########### epoch 9 ###########
########### loop 1250 ###########
test loss:   1.603232   test accuracy:   0.531250
########### loop 1250 ###########
train loss:   1.373057
train loss:   1.456507
train loss:   1.355876
train loss:   1.279042
train loss:   1.251114
train loss:   1.429201
train loss:   1.488788
train loss:   1.387632
train loss:   1.484648
train loss:   1.511462

...

train loss:   0.833263
train loss:   0.671910
train loss:   0.979138
train loss:   1.075457
train loss:   0.647659
train loss:   0.916961
train loss:   1.018179
train loss:   0.924337
train loss:   0.970736
train loss:   0.688175
train loss:   0.887796
train loss:   0.710184
train loss:   0.863547
train loss:   1.068855
train loss:   0.765688
train loss:   1.016616
train loss:   0.816044
train loss:   0.891372
train loss:   0.858094
train loss:   1.001485
train loss:   0.936107
train loss:   0.736645
train loss:   0.917469
train loss:   1.037493
train loss:   0.776014
train loss:   0.925695
########### epoch 90 ###########
########### loop 12600 ###########
test loss:   1.075247   test accuracy:   0.687500
########### loop 12600 ###########
train loss:   0.764920
train loss:   0.663758
train loss:   0.836730
train loss:   0.955207
train loss:   0.763048
train loss:   0.806202
train loss:   1.237902
train loss:   0.894196
train loss:   0.828032
train loss:   1.049451
train loss:   0.974

train loss:   0.909694
train loss:   0.898664
train loss:   0.875248
train loss:   0.862255
train loss:   0.704239
train loss:   1.086246
train loss:   0.710809
train loss:   0.891441
train loss:   0.764549
########### epoch 92 ###########
########### loop 12900 ###########
test loss:   0.992355   test accuracy:   0.718750
########### loop 12900 ###########
train loss:   0.959140
train loss:   0.663508
train loss:   0.801085
train loss:   0.800201
train loss:   0.979393
train loss:   0.701811
train loss:   0.713571
train loss:   0.757094
train loss:   0.746813
train loss:   0.878533
train loss:   0.703780
train loss:   0.787327
train loss:   0.829838
train loss:   0.729720
train loss:   0.757391
train loss:   0.887697
train loss:   0.923988
train loss:   0.611472
train loss:   0.998790
train loss:   0.750497
train loss:   0.658039
train loss:   0.763397
train loss:   0.872304
train loss:   0.945134
train loss:   0.804786
train loss:   0.842463
train loss:   0.929865
train loss:   0.593

train loss:   0.807653
train loss:   0.675535
train loss:   0.752820
train loss:   0.885379
train loss:   0.925384
train loss:   0.851331
train loss:   0.959152
train loss:   0.915245
train loss:   0.534908
train loss:   0.746237
train loss:   0.799546
train loss:   0.813680
train loss:   0.892035
train loss:   0.505484
train loss:   0.795403
train loss:   0.748797
train loss:   0.918008
train loss:   0.673452
train loss:   0.764942
train loss:   0.844600
train loss:   0.933982
train loss:   1.034437
train loss:   0.877009
train loss:   0.639627
train loss:   0.902506
train loss:   1.013747
train loss:   0.672384
train loss:   1.090391
train loss:   0.865808
train loss:   0.734631
train loss:   0.880699
train loss:   0.829412
train loss:   0.679255
train loss:   0.939772
train loss:   0.858034
train loss:   0.788312
train loss:   0.756759
train loss:   0.578433
train loss:   0.573434
train loss:   0.807617
train loss:   0.787691
train loss:   0.866290
train loss:   1.026126
train loss:

train loss:   0.775122
train loss:   0.715544
train loss:   0.525004
train loss:   0.534121
train loss:   0.717647
train loss:   0.756962
train loss:   0.831072
train loss:   0.912913
train loss:   0.768660
train loss:   0.803724
train loss:   0.977458
train loss:   1.105895
train loss:   1.061079
train loss:   1.067332
train loss:   0.774289
train loss:   0.881088
train loss:   0.878086
train loss:   0.769823
train loss:   0.645890
train loss:   0.743107
train loss:   0.773221
train loss:   0.696291
train loss:   0.920272
train loss:   0.721011
train loss:   0.782108
train loss:   0.634266
train loss:   0.889846
train loss:   0.839816
train loss:   0.820718
train loss:   0.930561
train loss:   0.830510
train loss:   0.757863
########### epoch 97 ###########
########### loop 13550 ###########
test loss:   1.288317   test accuracy:   0.625000
########### loop 13550 ###########
train loss:   0.753659
train loss:   0.679012
train loss:   0.641622
train loss:   1.009590
train loss:   0.949

train loss:   0.990360
train loss:   0.876682
train loss:   0.775984
train loss:   0.708000
train loss:   0.904588
train loss:   0.683431
train loss:   0.739449
train loss:   0.707449
train loss:   0.741410
train loss:   0.671711
train loss:   1.037302
train loss:   1.099668
train loss:   0.633414
train loss:   0.983492
train loss:   0.984274
########### epoch 99 ###########
########### loop 13850 ###########
test loss:   0.968486   test accuracy:   0.750000
########### loop 13850 ###########
train loss:   0.852460
train loss:   0.950079
train loss:   0.768620
train loss:   0.901880
train loss:   0.754170
train loss:   0.941260
train loss:   0.995813
train loss:   0.704735
train loss:   0.949708
train loss:   0.857927
train loss:   0.987699
train loss:   0.789526
train loss:   0.998022
train loss:   0.869377
train loss:   0.689916
train loss:   0.976279
train loss:   1.061355
train loss:   0.796331
train loss:   0.882178
train loss:   0.636997
train loss:   0.617916
train loss:   0.628

In [22]:
with tf.variable_scope('test1'):
    weights = tf.get_variable('W', [1], trainable = False)

ValueError: Variable test1/W already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "D:\Users\XX\Anaconda2\envs\gpu-env-tf\lib\site-packages\tensorflow\python\framework\ops.py", line 1204, in __init__
    self._traceback = self._graph._extract_stack()  # pylint: disable=protected-access
  File "D:\Users\XX\Anaconda2\envs\gpu-env-tf\lib\site-packages\tensorflow\python\framework\ops.py", line 2628, in create_op
    original_op=self._default_original_op, op_def=op_def)
  File "D:\Users\XX\Anaconda2\envs\gpu-env-tf\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 767, in apply_op
    op_def=op_def)


In [42]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    with tf.variable_scope('test1', reuse = True):
        var = tf.get_variable('W')
        print(var.name)
        print(var.eval())
        sess.run(var.assign([111]))
        print(var.eval())
        print(sess.run(var))
        np.savez('own_last_layer_weights.npz', xxx = [sess.run(var)])
        
        
# 取值 和 sess的细节

SyntaxError: keyword can't be an expression (<ipython-input-42-da1db3f03447>, line 10)

In [39]:
weights_dict_test = np.load('own_last_layer_weights.npz', encoding = 'bytes')

In [41]:
print(weights_dict_test.keys())
print(weights_dict_test['xxx'])

['xxx']
[[111.]]
